In [ ]:
#!pip install tslearn
#!pip install sktime
import pandas as pd
import numpy as np
import pathlib
from tqdm.notebook import tqdm
import librosa
import awkward as ak
import matplotlib.pyplot as plt
from IPython.display import Audio 
from scipy.signal import decimate
import tslearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from tslearn.piecewise import PiecewiseAggregateApproximation
from tslearn.piecewise import SymbolicAggregateApproximation
from tslearn.piecewise import OneD_SymbolicAggregateApproximation
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.transformations.panel.rocket import MiniRocket
from pyts.classification import KNeighborsClassifier

In [ ]:
X_train= np.load("RavdessAudioOnlyNumpy__X_train_compresso.npy")
X_test = np.load("RavdessAudioOnlyNumpy__X_test_compresso.npy")
y_train= pd.read_csv("RavdessAudioOnlyNumpy__Y_train.csv")
y_test= pd.read_csv("RavdessAudioOnlyNumpy__Y_test.csv")

start_index = 5000
end_index = 13000

selected_intervals = []
selected_intervals2 = []

for i in range(len(X_train)):
    selected_interval = X_train[i, start_index:end_index]
    selected_intervals.append(selected_interval)

selected_intervals = np.array(selected_intervals)

for i in range(len(X_test)):
    selected_interval2 = X_test[i, start_index:end_index]
    selected_intervals2.append(selected_interval2)

selected_intervals2 = np.array(selected_intervals2)

X_train= selected_intervals
X_test= selected_intervals2

In [ ]:
from tslearn.preprocessing import TimeSeriesScalerMinMax
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train).reshape(X_train.shape[0], X_train.shape[1])
X_test = scaler.fit_transform(X_test).reshape(X_test.shape[0], X_test.shape[1])

In [ ]:
sax = SymbolicAggregateApproximation(n_segments=1000, alphabet_size_avg= 50)
X_train_sax = np.squeeze(sax.fit_transform(X_train))
X_test_sax = np.squeeze(sax.transform(X_test))

c:\Users\Alex\anaconda3\envs\myenv\lib\site-packages\tslearn\utils\utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 1828 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
c:\Users\Alex\anaconda3\envs\myenv\lib\site-packages\tslearn\utils\utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 624 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '


In [ ]:
y_train = y_train['vocal_channel']
y_test = y_test['vocal_channel']

In [ ]:
from pyts.classification import KNeighborsClassifier
from sklearn.metrics import classification_report

clf = KNeighborsClassifier(metric='dtw')
clf.fit(X_train_sax, y_train)
y_pred = clf.predict(X_test_sax)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        song       0.73      0.69      0.71       264
      speech       0.78      0.81      0.80       360

    accuracy                           0.76       624
   macro avg       0.75      0.75      0.75       624
weighted avg       0.76      0.76      0.76       624



In [ ]:
clf = KNeighborsClassifier(metric='dtw_sakoechiba')
clf.fit(X_train_sax, y_train)
y_pred = clf.predict(X_test_sax)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        song       0.72      0.79      0.75       264
      speech       0.83      0.78      0.80       360

    accuracy                           0.78       624
   macro avg       0.78      0.78      0.78       624
weighted avg       0.78      0.78      0.78       624



In [ ]:
clf = KNeighborsClassifier(metric='dtw_itakura')
clf.fit(X_train_sax, y_train)
y_pred = clf.predict(X_test_sax)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        song       0.75      0.78      0.76       264
      speech       0.83      0.81      0.82       360

    accuracy                           0.80       624
   macro avg       0.79      0.79      0.79       624
weighted avg       0.80      0.80      0.80       624



In [ ]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999
0,0.468339,0.461069,0.471427,0.475357,0.475084,0.487562,0.487538,0.489554,0.485613,0.474879,...,0.478964,0.478972,0.478954,0.479004,0.479027,0.478991,0.478989,0.478963,0.478963,0.478972
1,0.485669,0.493362,0.490341,0.488976,0.491020,0.488943,0.489061,0.488546,0.488852,0.481916,...,0.487636,0.487621,0.487634,0.487625,0.487627,0.487639,0.487655,0.487680,0.487670,0.487687
2,0.564543,0.409476,0.280079,0.288848,0.343170,0.391479,0.497179,0.609978,0.607220,0.512948,...,0.427856,0.427857,0.427860,0.427847,0.427886,0.427911,0.427853,0.427855,0.427861,0.427853
3,0.239673,0.253786,0.263509,0.213520,0.228931,0.189220,0.276660,0.628976,0.737018,0.635710,...,0.396614,0.396620,0.396589,0.396599,0.396593,0.396596,0.396626,0.396602,0.396591,0.396599
4,0.515349,0.514042,0.516659,0.517696,0.514238,0.514163,0.512698,0.506910,0.508050,0.506850,...,0.511319,0.511196,0.511307,0.511688,0.512238,0.511792,0.511950,0.511555,0.511141,0.511186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,0.521492,0.607316,0.659782,0.646481,0.578757,0.473224,0.361634,0.282611,0.224281,0.183593,...,0.494605,0.494436,0.494472,0.494530,0.495231,0.496948,0.496705,0.494615,0.494441,0.494834
1824,0.503376,0.503329,0.510438,0.504336,0.511172,0.514464,0.504678,0.505358,0.500509,0.510565,...,0.506371,0.506207,0.506324,0.506574,0.507017,0.507052,0.506807,0.507000,0.506998,0.506789
1825,0.465330,0.456811,0.455482,0.463061,0.476738,0.486637,0.490876,0.483626,0.474941,0.459031,...,0.468074,0.468971,0.468944,0.467587,0.467069,0.467501,0.467229,0.468269,0.469599,0.468780
1826,0.463317,0.481710,0.491810,0.496304,0.507003,0.500364,0.507207,0.514103,0.511075,0.511286,...,0.411781,0.344276,0.396192,0.357398,0.400889,0.541947,0.520326,0.521368,0.607204,0.565718
